In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [4]:
import folium
from geopy.geocoders import Nominatim

In [5]:
# Note:i am submitting the assignment in a single link to Github so i split the assignment in 3 parts in this Notebook

# Part 1

In [6]:
# i use BeuitifulSoup and requests libirary to extrat data from the Wikipedia of Canada

In [7]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto = requests.get(url).text
soup = BeautifulSoup(toronto,'lxml')
table=soup.find('table',class_='wikitable sortable')


In [8]:
df_toronto= pd.read_html(str(table))[0]

In [9]:
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [34]:
#Lets rename the Postal code as mentioned in the assignment
df_toronto.rename(columns={'Postal Code':'Postal_Code'},inplace=True)

In [35]:
df_toronto.head()

,Postal_Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [36]:
#Lets drop the columns in Borough which is not assignmet
df_toronto.drop(df_toronto[df_toronto.Borough =='Not assigned'].index,inplace=True)

In [37]:
df_toronto.head()

,Postal_Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [38]:
df_toronto.shape

(103, 3)

# Part 2

In [39]:
# Lets create dataframe of lattitude and longitude from csv file
df_ll=pd.read_csv('https://cocl.us/Geospatial_data')
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
# Lets rename the postal Code
df_ll.rename(columns={'Postal Code':'Postal_Code'},inplace=True)
df_ll.head()

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [42]:
#Lets merge two Dataframe so we have to set index to postal code which is common in both dfs

df_toronto.set_index('Postal_Code')
df_ll.set_index('Postal_Code')
df_neigh = pd.merge(df_toronto,df_ll)

In [43]:
df_neigh.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [44]:
df_neigh.shape

(103, 5)

# Part 3

In [45]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_neigh['Latitude'], df_neigh['Longitude'], df_neigh['Borough'], df_neigh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [47]:
# Lets create a new Dataframe of North York by slicing the df_neigh for segement and cluster only in North York Neighborhood
df_north_york = df_neigh[df_neigh['Borough'] == 'North York'].reset_index(drop=True)

In [48]:
df_north_york.head()

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [49]:
#Lets get the geographical location of North York
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [50]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_north_york['Latitude'], df_north_york['Longitude'], df_north_york['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

In [51]:
CLIENT_ID = 'QUGEACJVSD5VBPK2M2C1OIVILC2WNEKESRJYF2KW4PMWIZNR' # your Foursquare ID
CLIENT_SECRET = 'JQDL1JXMAF3BAKS4MZOTQST1E33RXIJLXNBTUEA0M25VPD0B' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QUGEACJVSD5VBPK2M2C1OIVILC2WNEKESRJYF2KW4PMWIZNR
CLIENT_SECRET:JQDL1JXMAF3BAKS4MZOTQST1E33RXIJLXNBTUEA0M25VPD0B


In [53]:
#get the neighbourhood name
df_north_york.loc[0, 'Neighborhood']

'Parkwoods'

In [54]:
# gets Parkwoods Latitude and Longitude
park_latitude =df_north_york.loc[0,'Latitude']
park_longitude =df_north_york.loc[0,'Longitude']
neigh_name =df_north_york.loc[0,'Neighborhood']
print('The Latitude and Longitude of {},are {},{}'.format(neigh_name,park_latitude,park_longitude))

The Latitude and Longitude of Parkwoods,are 43.7532586,-79.3296565


In [51]:
# Now lets get top 100 venues that are in Parkwood around 500 mtrs

In [55]:
radius = 500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    park_latitude, 
    park_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=QUGEACJVSD5VBPK2M2C1OIVILC2WNEKESRJYF2KW4PMWIZNR&client_secret=JQDL1JXMAF3BAKS4MZOTQST1E33RXIJLXNBTUEA0M25VPD0B&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [56]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f0b172f0ab6c4151b695507'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [58]:
venues = result['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,TTC stop #8380,Bus Stop,43.752672,-79.326351
2,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [61]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


# Lets create a function to repeat the same process for neighborhood in North York


In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [64]:
north_york_venues = getNearbyVenues(names=df_north_york['Neighborhood'],
                                   latitudes=df_north_york['Latitude'],
                                   longitudes=df_north_york['Longitude']
                                  )


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [65]:
# Lets check how many venues in each Neighborhood
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Don Mills,27,27,27,27,27,27
Downsview,15,15,15,15,15,15
"Fairview, Henry Farm, Oriole",64,64,64,64,64,64
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


# Lets check how many unique venues in North York Neighborhood

In [66]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 101 uniques categories.


# Analyze each Neighborhood

In [67]:
# one hot encoding
northyork_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Lets group the rows by Neighborhood and by taking the mean of frequency of Occurance of each catageory

In [68]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.050000,0.050000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.083333,0.000000,0.041667,0.000000,0.041667,0.00,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.000000,0.037037,0.000000,0.074074,0.037037,0.000000,0.000000,...,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.015625,0.000000,0.000000,0.015625,0.000000,0.015625,0.031250,...,0.000000,0.000000,0.015625,0.000000,0.015625,0.015625,0.00,0.015625,0.000000,0.046875
6,Glencairn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


# Lets print each neighborhood along with 5 common venues

In [70]:
num_top_venues = 5

for hood in northyork_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = northyork_grouped[northyork_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0    Coffee Shop  0.10
1           Bank  0.10
2  Shopping Mall  0.05
3      Gift Shop  0.05
4    Gas Station  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.08
1      Sandwich Place  0.08
2         Coffee Shop  0.08
3  Italian Restaurant  0.08
4    Sushi Restaurant  0.08


----Don Mills----
              venue  freq
0               Gym  0.11
1        Restaurant  0.07
2  Asian Restaurant  0.07
3        Beer Store  0.07
4       Coffee Shop  0.07


----Downsview----
              venue  freq
0              Park  0.13
1     Grocery Store  0.13
2  Business Service  0.07
3      Liquor Store  0.07
4     Shopping Mall  0.07


----Fairview, Henry Farm, Oriole----
   

# Lets puts that into Dataframe

In [71]:
#First, let's write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [73]:
import numpy as np

# Lets create a dataframe displaying top 10 venue of easc neighborhood

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Mobile Phone Shop,Bridal Shop,Ice Cream Shop,Pharmacy,Diner,Sandwich Place,Shopping Mall,Pizza Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Distribution Center,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Restaurant,Sandwich Place,Coffee Shop,Sushi Restaurant,Italian Restaurant,Liquor Store,Café,Butcher,Juice Bar,Comfort Food Restaurant
3,Don Mills,Gym,Restaurant,Asian Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Discount Store,Chinese Restaurant,Dim Sum Restaurant,Athletics & Sports
4,Downsview,Grocery Store,Park,Liquor Store,Airport,Food Truck,Discount Store,Athletics & Sports,Gym / Fitness Center,Bank,Baseball Field
5,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Juice Bar,Japanese Restaurant,Food Court,Convenience Store,Shoe Store
6,Glencairn,Japanese Restaurant,Pizza Place,Bakery,Pub,Italian Restaurant,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
7,Hillcrest Village,Golf Course,Mediterranean Restaurant,Pool,Athletics & Sports,Dog Run,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
8,Humber Summit,Pizza Place,Italian Restaurant,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,"Humberlea, Emery",Baseball Field,Women's Store,Distribution Center,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [77]:
from sklearn.cluster import KMeans

# Cluster Neighborhood

In [82]:
# set number of clusters
kclusters = 3

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

In [84]:
# add clustering labels

northyork_merged = df_north_york

northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

northyork_merged.head() 

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Women's Store,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Boutique,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Bakery,Dog Run
3,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Restaurant,Asian Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Discount Store,Chinese Restaurant,Dim Sum Restaurant,Athletics & Sports
4,M6B,North York,Glencairn,43.709577,-79.445073,2.0,Japanese Restaurant,Pizza Place,Bakery,Pub,Italian Restaurant,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


# Cluster 1

In [106]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 0, northyork_merged.columns[[1] + list(range(3, northyork_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,43.753259,-79.329656,0.0,Park,Bus Stop,Food & Drink Shop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
12,North York,43.757490,-79.374714,0.0,0,0,0,0,0,0,0,0,0,0
22,North York,43.752758,-79.400049,0.0,Park,Convenience Store,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


# Cluster 2

In [107]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 1, northyork_merged.columns[[1] + list(range(3, northyork_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Women's Store,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,North York,43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Furniture / Home Store,Boutique,Event Space,Miscellaneous Shop,Coffee Shop,Vietnamese Restaurant,Bakery,Dog Run
3,North York,43.745906,-79.352188,1.0,Gym,Restaurant,Asian Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Discount Store,Chinese Restaurant,Dim Sum Restaurant,Athletics & Sports
5,North York,43.725900,-79.340923,1.0,Gym,Restaurant,Asian Restaurant,Japanese Restaurant,Beer Store,Coffee Shop,Discount Store,Chinese Restaurant,Dim Sum Restaurant,Athletics & Sports
6,North York,43.803762,-79.363452,1.0,Golf Course,Mediterranean Restaurant,Pool,Athletics & Sports,Dog Run,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
7,North York,43.754328,-79.442259,1.0,Bank,Coffee Shop,Mobile Phone Shop,Bridal Shop,Ice Cream Shop,Pharmacy,Diner,Sandwich Place,Shopping Mall,Pizza Place
8,North York,43.778517,-79.346556,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Restaurant,Juice Bar,Japanese Restaurant,Food Court,Convenience Store,Shoe Store
9,North York,43.767980,-79.487262,1.0,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Caribbean Restaurant,Miscellaneous Shop,Massage Studio,Bar,Diner,Comfort Food Restaurant,Construction & Landscaping
10,North York,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Distribution Center,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
11,North York,43.737473,-79.464763,1.0,Grocery Store,Park,Liquor Store,Airport,Food Truck,Discount Store,Athletics & Sports,Gym / Fitness Center,Bank,Baseball Field


# Cluster 3

In [111]:
northyork_merged.loc[northyork_merged['Cluster Labels'] == 2, northyork_merged.columns[[1] + list(range(3, northyork_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,43.709577,-79.445073,2.0,Japanese Restaurant,Pizza Place,Bakery,Pub,Italian Restaurant,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
15,North York,43.756303,-79.565963,2.0,Pizza Place,Italian Restaurant,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
